In [2]:
import os
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss

In [3]:
PATH              = "../data/"
TRAIN_F           = os.path.join(PATH, "train_features.csv")
TRAIN_T           = os.path.join(PATH, "train_targets_scored.csv")

In [105]:
nets = ["MoANet2", "LogisticRegression", "WideAndDeepLearning", "LinearReluBnDropout", "TablarNet"]

In [106]:
f = pd.read_csv(TRAIN_F)
t = pd.read_csv(TRAIN_T)[f['cp_type'] == "trt_cp"].reset_index(drop = True)
del f
gc.collect()
t

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21944,id_fffb1ceed,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21945,id_fffb70c0c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21946,id_fffcb9e7c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [134]:
predictions = []
for net in nets:
    PREDICTION = os.path.join(PATH, f"{net}-prediction.csv")
    predictions.append(pd.read_csv(PREDICTION))
for i, prediction in enumerate(predictions):
    predictions[i] = pd.DataFrame(t["sig_id"]).merge(prediction, how="inner", on="sig_id")

In [135]:
def logLoss(y_true, y_pred):
    score = 0
    for i in range(y_true.shape[1]):
        x = y_true[:, i]
        z = y_pred[:, i]
        score_ = log_loss(x, z)
        score += score_ / y_true.shape[1]

    return score

In [136]:
for i in range(len(predictions)):
    print(logLoss(t.iloc[:, 1:].to_numpy(), predictions[i].iloc[:, 1:].to_numpy()))

0.01731139480534198
0.01704148095252174
0.017469502491622227
0.29035027587195006
0.018206504571484216


In [57]:
def getLogLoss(p, w):
    pred = numpy()
    for prediction, weight in zip(p[1:], w[1:]):
        pred += prediction.iloc[:, 1:] * weight
        
    pred /= sum(w)
    
    return log_loss(t.iloc[:, 1:].to_numpy().reshape(-1, 1), pred.to_numpy().reshape(-1, 1))

In [81]:
for i in range(0, 5):
    z = 1 - i / 4
    w = [2.25, 1.75, 1.25, 0, 1]
    print(getLogLoss(predictions, w))

0.016103578354236803
0.016104433500685616
0.01611140674752546
0.016127969941986644
0.016164007192152777


In [59]:
w = [1, 1, 1, 1, 1]
print(getLogLoss(predictions, w))

0.05529589270151329


In [63]:
a = [1, 1, 0, 0]
b = [1, 1, 0, 0]
log_loss(a, b)

9.992007221626415e-16

In [83]:
w = [2.25, 1.75, 1.25, 0, 1]
submissions = []
for net in nets:
    SUBMISSION = os.path.join(PATH, f"{net}-submission.csv")
    submissions.append(pd.read_csv(SUBMISSION))

In [88]:
submission_final = submissions[0]
submission_final.iloc[:, 1:] = submission_final.iloc[:, 1:] * w[0]
for i in range(1, 5):
    submission_final.iloc[:, 1:] += submissions[i].iloc[:, 1:] * w[i]
submission_final.iloc[:, 1:] = submission_final.iloc[:, 1:] / sum(w)
submission_final

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.000200,0.001329,0.001421,0.013725,0.018096,0.004823,0.001087,0.006710,0.000139,...,0.000577,0.003545,0.003255,0.001044,0.000319,0.000925,0.000801,0.002405,0.004920,0.000879
1,id_001897cda,0.000031,0.000184,0.000685,0.001283,0.001661,0.002271,0.002103,0.007067,0.004381,...,0.000305,0.000830,0.001637,0.000169,0.013206,0.000181,0.007916,0.000304,0.001623,0.002027
2,id_002429b5b,0.000137,0.000051,0.002883,0.019022,0.019338,0.003014,0.004544,0.002735,0.000232,...,0.001299,0.000360,0.004151,0.011575,0.002670,0.000094,0.003059,0.006555,0.001061,0.001797
3,id_00276f245,0.000112,0.000081,0.001095,0.011610,0.009227,0.005220,0.001122,0.003018,0.000098,...,0.000254,0.000416,0.002213,0.013762,0.002968,0.000093,0.001803,0.000860,0.000959,0.001575
4,id_0027f1083,0.002058,0.000838,0.001953,0.009354,0.020107,0.001773,0.005833,0.002545,0.000148,...,0.000615,0.000381,0.004676,0.001531,0.001236,0.000724,0.001839,0.001312,0.000421,0.001082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,0.000212,0.000950,0.000588,0.001904,0.009704,0.001878,0.000440,0.001178,0.000327,...,0.000156,0.004903,0.001948,0.108665,0.004694,0.000473,0.004838,0.000641,0.001184,0.000890
3978,id_ff925dd0d,0.018992,0.005394,0.000331,0.003400,0.021987,0.004605,0.006798,0.002258,0.000213,...,0.000113,0.000901,0.001783,0.000932,0.001278,0.001448,0.002030,0.000715,0.000249,0.000571
3979,id_ffb710450,0.001046,0.000279,0.000353,0.005989,0.036731,0.005537,0.004380,0.003640,0.000164,...,0.000299,0.000406,0.001141,0.000242,0.001720,0.000135,0.001592,0.000512,0.000200,0.000584
3980,id_ffbb869f2,0.000805,0.000412,0.000464,0.017659,0.038962,0.007769,0.005673,0.001797,0.000359,...,0.000031,0.000134,0.002457,0.000350,0.000948,0.000059,0.001742,0.001716,0.000381,0.002664


In [89]:
submission_final.to_csv("submission.csv", index=False)

In [97]:
k = pd.DataFrame(np.ones((21948, 207)))
k

,0,1,2,3,4,5,6,7,8,9,...,197,198,199,200,201,202,203,204,205,206
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
21944,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
21945,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
21946,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [98]:
getLogLoss([k], [1])

34.41089924845396

In [4]:
f = pd.read_csv(TRAIN_F)

In [5]:
f

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
23810,id_fffb70c0c,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
23811,id_fffc1c3f4,ctl_vehicle,48,D2,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,...,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808
23812,id_fffcb9e7c,trt_cp,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,...,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740
